In [10]:
import numpy as np

In [24]:
import pandas as pd
import nltk
import csv

df = pd.read_csv('review_text.csv',encoding='utf8')
docs = df['reviews'].values
data = []
for doc in docs:
    tokens = nltk.word_tokenize(doc)
    pos = nltk.pos_tag(tokens)
    sentence = []
    for item in pos:
        words = {}
        words['token'] = item[0]
        words['pos'] = item[1]
        words['label'] = 'O' #Default BIO tag
        sentence.append(words)
    data.append(sentence)

In [28]:
print(len(data))
a = [2,3,4,5]
a[1:]

898


[3, 4, 5]

In [30]:
import json
with open('reviews-aspect-0-299.json', 'w') as outfile:
    json.dump(data[:300], outfile)
with open('reviews-aspect-300-599.json', 'w') as outfile:
    json.dump(data[300:600], outfile)
with open('reviews-aspect-600.json', 'w') as outfile:
    json.dump(data[600:], outfile)

### Open Words with BIO Notation Label

In [2]:
import json
with open('aspect-BIO.json') as f:
    labeled = json.load(f)

display(type(labeled))
data = []
for sentence in labeled:
    sen = []
    for item in sentence:
        tup = (item['token'],item['pos'],item['label'])
        sen.append(tup)
    data.append(sen)
display(data)

list

[[('Despite', 'IN', 'O'),
  ('being', 'VBG', 'O'),
  ('set', 'VBN', 'O'),
  ('in', 'IN', 'O'),
  ('one', 'CD', 'O'),
  ('of', 'IN', 'O'),
  ('those', 'DT', 'O'),
  ('large', 'JJ', 'O'),
  ('American', 'JJ', 'O'),
  ('style', 'NN', 'O'),
  ('outdoor', 'MD', 'O'),
  ('home', 'VB', 'O'),
  ('centre', 'NN', 'O'),
  ('shopping', 'NN', 'O'),
  ('malls', 'NNS', 'O'),
  (',', ',', 'O'),
  ('XS', 'NNP', 'B'),
  ('Espresso', 'NNP', 'I'),
  ('manages', 'VBZ', 'O'),
  ('to', 'TO', 'O'),
  ('maintain', 'VB', 'O'),
  ('a', 'DT', 'O'),
  ('funky', 'JJ', 'O'),
  ('vibe', 'NN', 'O'),
  ('.', '.', 'O')],
 [('The', 'DT', 'O'),
  ('other', 'JJ', 'O'),
  ('upside', 'NN', 'O'),
  ('is', 'VBZ', 'O'),
  ('that', 'IN', 'O'),
  ('there', 'EX', 'O'),
  ('is', 'VBZ', 'O'),
  ('plenty', 'NN', 'O'),
  ('of', 'IN', 'O'),
  ('parking', 'NN', 'B'),
  ('.', '.', 'O')],
 [('Food', 'NN', 'B'),
  ('is', 'VBZ', 'O'),
  ('fresh', 'JJ', 'O'),
  ('and', 'CC', 'O'),
  ('colourful', 'JJ', 'O'),
  (',', ',', 'O'),
  ('and', 'CC'

In [3]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    features = [
        'bias',
        'word.lower=' + word.lower(),
        'postag=' + postag
    ]

    # Fitur untuk kata non awal kalimat
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:postag=' + postag1
        ])
    else:
        features.append('BOS') # Tanda awal kalimat

    # Fitur untuk kata non akhir kalimat
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:postag=' + postag1
        ])
    else:
        features.append('EOS') #tanda akhir kalimat
    return features

In [44]:
from sklearn.model_selection import train_test_split

def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

In [64]:
print(y)

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O'], ['B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'B', 'I', 'O'], ['O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'B', 'O', 'O', 'O'], ['B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'B', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O'], ['O', 'O', 'O', 'B', 'I', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'B', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O'], ['O', 'O', 'O',

### Get Split Index

In [61]:
kf = KFold(n_splits=10)
train_split = []
test_split = []
for train_index, test_index in kf.split(X):
    train_split.append(train_index)
    test_split.append(test_index)

10

In [84]:
y_train = [y[j] for j in train_split[2]]
print(y_train)

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O'], ['B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'B', 'I', 'O'], ['O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'B', 'O', 'O', 'O'], ['B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'B', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O'], ['O', 'O', 'O', 'B', 'I', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'B', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O'], ['O', 'O', 'O',

## Generate Model

In [101]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)
for i in range(len(train_split)):
    X_train = [X[j] for j in train_split[i]]
    y_train = [y[j] for j in train_split[i]]
    for xseq, yseq in zip(X_train, y_train):
        trainer.append(xseq, yseq)
    trainer.set_params({
        'c1': 0.1, #L1 penalty
        'c2': 0.01, #L2 penalty
        'max_iterations': 200,
        'feature.possible_transitions': True
    })
    trainer.train('crf.model_'+str(i))

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 5143
Seconds required: 0.017

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 2965.432524
Feature norm: 1.000000
Error norm: 2045.092997
Active features: 5054
Line search trials: 1
Line search step: 0.000126
Seconds required for this iteration: 0.005

***** Iteration #2 *****
Loss: 2537.814600
Feature norm: 1.345533
Error norm: 717.591510
Active features: 4811
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #3 *****
Loss: 2433.836074
Feature norm: 1.471054
Error norm: 658.487876
Active features: 4984
Line search trials: 1
Line search step: 1.000000
Seconds required for this iterat

***** Iteration #64 *****
Loss: 425.563237
Feature norm: 61.669945
Error norm: 9.371089
Active features: 1529
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.004

***** Iteration #65 *****
Loss: 425.273723
Feature norm: 61.638014
Error norm: 5.883524
Active features: 1526
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #66 *****
Loss: 425.038561
Feature norm: 61.672179
Error norm: 10.202414
Active features: 1526
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #67 *****
Loss: 424.836567
Feature norm: 61.660888
Error norm: 6.426531
Active features: 1520
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.005

***** Iteration #68 *****
Loss: 424.671978
Feature norm: 61.703079
Error norm: 5.631767
Active features: 1518
Line search trials: 1
Line search step: 1.000000
Seconds required for this it

***** Iteration #132 *****
Loss: 420.703371
Feature norm: 62.757163
Error norm: 5.904290
Active features: 1405
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.006

***** Iteration #133 *****
Loss: 420.699758
Feature norm: 62.754059
Error norm: 10.860874
Active features: 1400
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #134 *****
Loss: 420.636843
Feature norm: 62.788328
Error norm: 8.767546
Active features: 1402
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #135 *****
Loss: 420.594042
Feature norm: 62.786919
Error norm: 4.784902
Active features: 1397
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.005

***** Iteration #136 *****
Loss: 420.578965
Feature norm: 62.796829
Error norm: 7.922965
Active features: 1395
Line search trials: 1
Line search step: 1.000000
Seconds required for th

***** Iteration #178 *****
Loss: 419.664696
Feature norm: 62.796050
Error norm: 5.685951
Active features: 1370
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.005

***** Iteration #179 *****
Loss: 419.643254
Feature norm: 62.783148
Error norm: 1.627185
Active features: 1367
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.006

***** Iteration #180 *****
Loss: 419.633379
Feature norm: 62.782254
Error norm: 5.126744
Active features: 1365
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.006

***** Iteration #181 *****
Loss: 419.615035
Feature norm: 62.775208
Error norm: 1.709710
Active features: 1366
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.007

***** Iteration #182 *****
Loss: 419.607436
Feature norm: 62.778145
Error norm: 5.635787
Active features: 1363
Line search trials: 2
Line search step: 0.500000
Seconds required for thi

***** Iteration #27 *****
Loss: 879.681113
Feature norm: 54.338634
Error norm: 102.529252
Active features: 3079
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.009

***** Iteration #28 *****
Loss: 845.141286
Feature norm: 57.526663
Error norm: 94.116876
Active features: 3045
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.008

***** Iteration #29 *****
Loss: 817.127222
Feature norm: 59.757006
Error norm: 54.484396
Active features: 3000
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.006

***** Iteration #30 *****
Loss: 789.277092
Feature norm: 61.522940
Error norm: 23.894907
Active features: 2985
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.006

***** Iteration #31 *****
Loss: 771.314557
Feature norm: 63.901221
Error norm: 142.431298
Active features: 2941
Line search trials: 1
Line search step: 1.000000
Seconds required for t

...

***** Iteration #125 *****
Loss: 1586.285252
Feature norm: 126.395280
Error norm: 27.481482
Active features: 2053
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.026

***** Iteration #126 *****
Loss: 1586.060458
Feature norm: 126.492846
Error norm: 41.139715
Active features: 2054
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.025

***** Iteration #127 *****
Loss: 1585.687643
Feature norm: 126.512719
Error norm: 23.918797
Active features: 2052
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.025

***** Iteration #128 *****
Loss: 1585.586112
Feature norm: 126.606606
Error norm: 47.434059
Active features: 2050
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.023

***** Iteration #129 *****
Loss: 1585.112501
Feature norm: 126.629651
Error norm: 23.533988
Active features: 2052
Line search trials: 1
Line search step: 1.000000
Seconds r

***** Iteration #195 *****
Loss: 1572.123734
Feature norm: 128.079711
Error norm: 19.252491
Active features: 1979
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.023

***** Iteration #196 *****
Loss: 1572.082672
Feature norm: 128.101167
Error norm: 33.390830
Active features: 1981
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.022

***** Iteration #197 *****
Loss: 1571.850192
Feature norm: 128.094988
Error norm: 20.053754
Active features: 1979
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.023

***** Iteration #198 *****
Loss: 1571.823815
Feature norm: 128.115022
Error norm: 34.848963
Active features: 1976
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.023

***** Iteration #199 *****
Loss: 1571.561523
Feature norm: 128.102166
Error norm: 16.264143
Active features: 1975
Line search trials: 1
Line search step: 1.000000
Seconds r

***** Iteration #47 *****
Loss: 1907.905048
Feature norm: 116.904892
Error norm: 170.847330
Active features: 2788
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.024

***** Iteration #48 *****
Loss: 1896.213693
Feature norm: 117.551215
Error norm: 138.550951
Active features: 2784
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.023

***** Iteration #49 *****
Loss: 1886.066484
Feature norm: 117.950971
Error norm: 114.797386
Active features: 2752
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.024

***** Iteration #50 *****
Loss: 1875.441891
Feature norm: 118.507841
Error norm: 170.877798
Active features: 2689
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.025

***** Iteration #51 *****
Loss: 1866.203094
Feature norm: 118.890816
Error norm: 168.437055
Active features: 2435
Line search trials: 1
Line search step: 1.000000
Seconds r

***** Iteration #115 *****
Loss: 1707.333969
Feature norm: 127.508979
Error norm: 67.172561
Active features: 2101
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.024

***** Iteration #116 *****
Loss: 1706.533371
Feature norm: 127.540722
Error norm: 35.997036
Active features: 2097
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.024

***** Iteration #117 *****
Loss: 1706.297719
Feature norm: 127.651157
Error norm: 61.625149
Active features: 2097
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.025

***** Iteration #118 *****
Loss: 1705.606775
Feature norm: 127.671221
Error norm: 35.531443
Active features: 2098
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.024

***** Iteration #119 *****
Loss: 1705.362696
Feature norm: 127.774843
Error norm: 60.844591
Active features: 2095
Line search trials: 1
Line search step: 1.000000
Seconds r

***** Iteration #193 *****
Loss: 1686.728950
Feature norm: 130.607024
Error norm: 41.093140
Active features: 2025
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.025

***** Iteration #194 *****
Loss: 1686.379753
Feature norm: 130.593270
Error norm: 18.749347
Active features: 2022
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.025

***** Iteration #195 *****
Loss: 1686.379274
Feature norm: 130.626299
Error norm: 39.852934
Active features: 2020
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.024

***** Iteration #196 *****
Loss: 1686.056086
Feature norm: 130.618860
Error norm: 17.065731
Active features: 2022
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.023

***** Iteration #197 *****
Loss: 1686.045477
Feature norm: 130.654555
Error norm: 36.341894
Active features: 2021
Line search trials: 1
Line search step: 1.000000
Seconds r

## Model Evaluation

In [104]:
from sklearn.metrics import classification_report

labels = {"B": 0, "I": 1,"O":2}
target_names = ['B', 'I','O']

for i in range(len(train_split)):
    tagger = pycrfsuite.Tagger()
    tagger.open('crf.model_'+str(i))
    X_test = [X[j] for j in test_split[i]]
    y_test = [y[j] for j in test_split[i]]
    y_pred = [tagger.tag(xseq) for xseq in X_test]
#     for iterr, X_test_item in enumerate(X_test):
#         for idx,word in enumerate(X_test_item):
#             print(word[1]+" - "+y_test[iterr][idx]+" - "+y_pred[iterr][idx])
    truth = np.array([labels[bio] for sentence in y_test for bio in sentence])
    prediction = np.array([labels[bio] for sentence in y_pred for bio in sentence])
    print(classification_report(truth, prediction, target_names=target_names))

              precision    recall  f1-score   support

           B       0.77      0.49      0.60        82
           I       0.55      0.12      0.20        48
           O       0.91      0.99      0.95       832

   micro avg       0.90      0.90      0.90       962
   macro avg       0.74      0.53      0.58       962
weighted avg       0.88      0.90      0.88       962

              precision    recall  f1-score   support

           B       0.89      0.89      0.89        45
           I       0.95      1.00      0.98        21
           O       0.99      0.99      0.99       722

   micro avg       0.99      0.99      0.99       788
   macro avg       0.95      0.96      0.95       788
weighted avg       0.99      0.99      0.99       788

              precision    recall  f1-score   support

           B       0.98      0.93      0.96        59
           I       1.00      1.00      1.00         9
           O       0.99      1.00      1.00       751

   micro avg       0